In [1]:
from lib.utils import *
import torch.backends.cudnn as cudnn
import torch
from lib.models import SRResNet,Generator
from lib.dataloaders import SRDataset_v2,SRDataset
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import pandas as pd
from lib.niqe.niqe import niqe
from lib.testmodel import get_test_model


In [2]:
# 设备参数
#device= torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ngpu = 1           # 用来运行的gpu数量

In [5]:
models,models_name=get_test_model(device)

In [6]:
def eval_SRDataset(models:list):
    # 测试集目录
    data_folder = "./data/"
    test_data_names = ["Set5","Set14", "BSD100"]
    
    for test_data_name in test_data_names:
        print("\n数据集 %s:\n" % test_data_name)

        # 定制化数据加载器
        test_dataset = SRDataset(data_folder,
                                split='test',
                                crop_size=0,
                                scaling_factor=4,
                                lr_img_type='imagenet-norm',
                                hr_img_type='[-1, 1]',
                                test_data_name=test_data_name)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=1,
                                                pin_memory=True)
 
        # 记录每个模型的 PSNR 和 SSIM值
        PSNRs = [AverageMeter() for i in range(0,len(models))]
        SSIMs = [AverageMeter() for i in range(0,len(models))]
        NIQEs = [AverageMeter() for i in range(0,len(models))]

        with torch.no_grad():
            # 逐批样本进行推理计算
            for i, (lr_imgs, hr_imgs) in enumerate(test_loader):
                
                # 数据移至默认设备
                lr_imgs = lr_imgs.to(device)  # (batch_size (1), 3, w / 4, h / 4), imagenet-normed
                hr_imgs = hr_imgs.to(device)  # (batch_size (1), 3, w, h), in [-1, 1]
 
                # 前向传播.
                sr_imgs=[]
                for i in range(0,len(models)):
                    sr_imgs=(models[i](lr_imgs))  # (1, 3, w, h), in [-1, 1]                
                    #计算NIQE
                    sr_imgs_PIL=convert_image(sr_imgs.squeeze(0).cpu().detach(),source='[-1, 1]',target='pil')
                    img = np.array(sr_imgs_PIL.convert('LA'))[:,:,0] # ref
                    niqe_score=np.float64(niqe(img))
                    NIQEs[i].update(niqe_score, lr_imgs.size(0))
                    


                    # 计算 PSNR 和 SSIM
                    sr_imgs_y = convert_image(sr_imgs, source='[-1, 1]', target='y-channel').squeeze(
                        0)  # (w, h), in y-channel
                    hr_imgs_y = convert_image(hr_imgs, source='[-1, 1]', target='y-channel').squeeze(0)  # (w, h), in y-channel
                    psnr = peak_signal_noise_ratio(hr_imgs_y.cpu().numpy(), sr_imgs_y.cpu().numpy(),
                                                data_range=255.)
                    ssim = structural_similarity(hr_imgs_y.cpu().numpy(), sr_imgs_y.cpu().numpy(),
                                                data_range=255.)
                    PSNRs[i].update(psnr, lr_imgs.size(0))
                    SSIMs[i].update(ssim, lr_imgs.size(0))

                    #print(type(psnr))
                    #print(type(niqe_score))
                    #return
        score=pd.DataFrame(columns=['PSNR','SSIM','NIQE'])
        for i in range(0,len(models)):
            score=score.append(pd.DataFrame([  [PSNRs[i].avg, SSIMs[i].avg, NIQEs[i].avg]  ],index=[models_name[i]],columns=['PSNR','SSIM','NIQE']))
            
        display(score)
 
    print("\n")

In [7]:
eval_SRDataset(models)


数据集 Set5:



,PSNR,SSIM,NIQE
srresnet,31.483523,0.895942,37.411470
srgan_sample,29.518227,0.862035,26.078771
srgan,29.020984,0.839001,21.970838
srresnet_v2,25.001936,0.794002,26.879233
srgan_v2_sample,18.723419,0.719465,32.190977
srgan_v2,16.726911,0.536544,51.455851
bicubic,28.431096,0.823250,37.877038



数据集 Set14:



,PSNR,SSIM,NIQE
srresnet,28.268789,0.790680,23.362103
srgan_sample,27.067182,0.760710,22.371134
srgan,25.651867,0.693234,12.312346
srresnet_v2,23.472897,0.713840,21.299658
srgan_v2_sample,18.805722,0.644872,22.589707
srgan_v2,17.693468,0.509123,40.167369
bicubic,26.101534,0.721988,22.606124



数据集 BSD100:



,PSNR,SSIM,NIQE
srresnet,27.352744,0.747913,19.635913
srgan_sample,26.374011,0.718243,22.798362
srgan,24.651668,0.643506,12.919538
srresnet_v2,24.261730,0.694956,18.896580
srgan_v2_sample,19.053461,0.611803,20.853981
srgan_v2,18.284762,0.503871,46.191330
bicubic,25.961409,0.685871,22.277370


In [8]:
def eval_SRDataset_v2(models:list):
    # 测试集目录
    data_folder = "./data/"
    test_data_names = ["Set5","Set14", "BSD100"]
    
    for test_data_name in test_data_names:
        print("\n数据集 %s:\n" % test_data_name)

        # 定制化数据加载器
        test_dataset = SRDataset_v2(data_folder,
                                split='test',
                                crop_size=0,
                                scaling_factor=4,
                                lr_img_type='imagenet-norm',
                                hr_img_type='[-1, 1]',
                                test_data_name=test_data_name,
                                gasuss_noise_sigma=0.001)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=1,
                                                pin_memory=True)
 
        # 记录每个模型的 PSNR 和 SSIM值
        PSNRs = [AverageMeter() for i in range(0,len(models))]
        SSIMs = [AverageMeter() for i in range(0,len(models))]
        NIQEs = [AverageMeter() for i in range(0,len(models))]

        with torch.no_grad():
            # 逐批样本进行推理计算
            for i, (lr_imgs, hr_imgs) in enumerate(test_loader):
                
                # 数据移至默认设备
                lr_imgs = lr_imgs.to(device)  # (batch_size (1), 3, w / 4, h / 4), imagenet-normed
                hr_imgs = hr_imgs.to(device)  # (batch_size (1), 3, w, h), in [-1, 1]
 
                # 前向传播.
                sr_imgs=[]
                for i in range(0,len(models)):
                    sr_imgs=(models[i](lr_imgs))  # (1, 3, w, h), in [-1, 1]                

                    
                    #计算NIQE
                    sr_imgs_PIL=convert_image(sr_imgs.squeeze(0).cpu().detach(),source='[-1, 1]',target='pil')
                    img = np.array(sr_imgs_PIL.convert('LA'))[:,:,0] # ref
                    niqe_score=np.float64(niqe(img))
                    NIQEs[i].update(niqe_score, lr_imgs.size(0))
                    


                    # 计算 PSNR 和 SSIM
                    sr_imgs_y = convert_image(sr_imgs, source='[-1, 1]', target='y-channel').squeeze(
                        0)  # (w, h), in y-channel
                    hr_imgs_y = convert_image(hr_imgs, source='[-1, 1]', target='y-channel').squeeze(0)  # (w, h), in y-channel
                    psnr = peak_signal_noise_ratio(hr_imgs_y.cpu().numpy(), sr_imgs_y.cpu().numpy(),
                                                data_range=255.)
                    ssim = structural_similarity(hr_imgs_y.cpu().numpy(), sr_imgs_y.cpu().numpy(),
                                                data_range=255.)
                    PSNRs[i].update(psnr, lr_imgs.size(0))
                    SSIMs[i].update(ssim, lr_imgs.size(0))

        score=pd.DataFrame(columns=['PSNR','SSIM','NIQE'])
        for i in range(0,len(models)):
            score=score.append(pd.DataFrame([  [PSNRs[i].avg, SSIMs[i].avg, NIQEs[i].avg]  ],index=[models_name[i]],columns=['PSNR','SSIM','NIQE']))
            
        display(score)
 
    print("\n")

In [9]:
eval_SRDataset_v2(models)


数据集 Set5:



,PSNR,SSIM,NIQE
srresnet,26.237078,0.725662,30.543297
srgan_sample,25.305810,0.694358,28.065622
srgan,25.665520,0.701413,39.453615
srresnet_v2,27.600361,0.787449,26.478580
srgan_v2_sample,18.045753,0.630439,29.525457
srgan_v2,16.103610,0.455537,57.588869
bicubic,25.944629,0.720306,34.824322



数据集 Set14:



,PSNR,SSIM,NIQE
srresnet,24.790782,0.636624,23.971610
srgan_sample,24.117581,0.609079,23.852659
srgan,24.377122,0.616132,21.449667
srresnet_v2,25.979028,0.682062,20.664624
srgan_v2_sample,18.199740,0.556134,21.276550
srgan_v2,17.283585,0.429066,47.216144
bicubic,24.467986,0.625982,22.455286



数据集 BSD100:



,PSNR,SSIM,NIQE
srresnet,24.857318,0.601364,21.884008
srgan_sample,24.323262,0.577336,23.404577
srgan,24.551729,0.584802,19.127575
srresnet_v2,25.570468,0.636726,19.572485
srgan_v2_sample,18.586232,0.521973,20.343026
srgan_v2,17.892046,0.419313,53.877190
bicubic,24.656468,0.592595,21.229618


In [4]:
imgDir='./results/bird/'
input='bird.jpg'
output='bird_x4'
#img = Image.open(imgDir+'bird_x4_realesrgan.jpg', mode='r')

score=super_revolution(models,models_name,imgDir,input,output,device)
#realesrgan_NIQE=get_NIQE_from_PIL(img,device)
#score=score.append(pd.DataFrame([realesrgan_NIQE],index=['realesrgan'],columns=['NIQE']))
display(score)

<built-in method ndimension of Tensor object at 0x7f4e5dfa9a40>


AssertionError: 

In [6]:
imgDir='./results/cat/'
input='cat.jpg'
output='cat_x4'
#img = Image.open(imgDir+'cat_x4_realesrgan.jpg', mode='r')

score=super_revolution(models,models_name,imgDir,input,output,device)
#realesrgan_NIQE=get_NIQE_from_PIL(img,device)
#score=score.append(pd.DataFrame([realesrgan_NIQE],index=['realesrgan'],columns=['NIQE']))


In [7]:
display(score)

,NIQE
srresnet,14.699076
srgan_sample,14.542596
srgan,11.489096
srresnet_v2,13.461814
srgan_v2_sample,15.064588
srgan_v2,29.428685
bicubic,14.765079


In [3]:
imgDir='./results/cat/'
img = Image.open(imgDir+'cat_x4_realesrgan.jpg', mode='r')

print(get_NIQE_from_PIL(img,device))


10.717857146991156


In [13]:

imgDir='./results/shark/'
input='shark_croped.jpg'
output='shark_croped_x4'
#img = Image.open(imgDir+'shark_croped_x4_realesrgan.jpg', mode='r')

score=super_revolution(models,models_name,imgDir,input,output,device)
#realesrgan_NIQE=get_NIQE_from_PIL(img,device)
#score=score.append(pd.DataFrame([realesrgan_NIQE],index=['realesrgan'],columns=['NIQE']))
display(score)